In [1]:
from __future__ import division
from __future__ import print_function

import sys, os
sys.path.append('../')

import tensorflow as tf
import numpy as np
import gc

from models.neural_network import NeuralNetwork
from models.fcc import FCC
from models.util import *


os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#Seed used for all calculations of training and test point indices 
SEED = 17

Using TensorFlow backend.


In [2]:
#Define params of model
input_shape = (545333)
num_classes = 2
eps = 10

In [3]:
#Load model from disk (Note: Drebin takes a few minutes to load the dataset)
model_name = 'DREBIN'
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model = FCC(model_name=model_name, dataset='drebin', seed=SEED)
print ('Total Model Params: %d' % model.num_params)
model.load_model(model_save_path, weights_save_path) 
#epochs = 10
#model.train(epochs=epochs)
#model.save_model(model_save_path, weights_save_path)   

Total Model Params: 34918082
Loaded model from disk


In [4]:
#Model Accuracy
print ('Model Accuracy: %.5f' % (model.model.evaluate(model.test_data, model.test_labels)[1]))

3913/3913 [==============================] - 14s 4ms/step
Model Accuracy: 0.98722


In [5]:
#Get training samples
num_train_samples = 100
data_indices = model.gen_rand_indices(low=0, high=model.train_data.shape[0], seed=SEED, num_samples=num_train_samples)
train_data = model.train_data[data_indices]
train_data_labels = model.train_labels[data_indices]
train_data_labels_int = np.argmax(train_data_labels, axis=1)

In [6]:
num_test_samples_per_class = 100
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.test_labels, seed=SEED, num_samples=num_test_samples_per_class)

#Get Regular, Noisy, FGSM, BIM, and CW test points
reg_data = model.test_data[test_indices]
noisy_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='NOISY', iterations=eps)
fgsm_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='FGSM')
bim_a_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='BIM-A', iterations=eps)
bim_b_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='BIM-B', iterations=eps)
#For JSMA we need to flip labels
jsma_tar = (model.test_labels[test_indices] == 0).astype(int)
jsma_data = model.generate_perturbed_data_binary(model.test_data[test_indices], jsma_tar,seed=SEED, perturbation='JSMA', iterations=eps)

In [7]:
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()
del model
gc.collect()
#Reload the model and weights
model = FCC(model_name=model_name, dataset='drebin', seed=SEED)
model.load_model(model_save_path, weights_save_path)    

Loaded model from disk


In [8]:
print ('Model Accuracy REG: %.5f' % (model.model.evaluate(reg_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy NOISY: %.5f' % (model.model.evaluate(noisy_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy FGSM: %.5f' % (model.model.evaluate(fgsm_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy BIM-A: %.5f' % (model.model.evaluate(bim_a_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy BIM-B: %.5f' % (model.model.evaluate(bim_b_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy JSMA: %.5f' % (model.model.evaluate(jsma_data,model.test_labels[test_indices])[1]))

200/200 [==============================] - 3s 17ms/step
Model Accuracy REG: 0.90000
200/200 [==============================] - 3s 16ms/step
Model Accuracy NOISY: 0.51000
200/200 [==============================] - 3s 15ms/step
Model Accuracy FGSM: 0.50000
200/200 [==============================] - 3s 16ms/step
Model Accuracy BIM-A: 0.00000
200/200 [==============================] - 3s 16ms/step
Model Accuracy BIM-B: 0.22000
200/200 [==============================] - 3s 15ms/step
Model Accuracy JSMA: 0.00000


In [9]:
#Get predictions
reg_preds = model.model.predict(reg_data.reshape(-1,input_shape))
noisy_preds = model.model.predict(noisy_data.reshape(-1,input_shape))
fgsm_preds = model.model.predict(fgsm_data.reshape(-1,input_shape))
bim_a_preds = model.model.predict(bim_a_data.reshape(-1,input_shape))
bim_b_preds = model.model.predict(bim_b_data.reshape(-1,input_shape))
jsma_preds = model.model.predict(jsma_data.reshape(-1,input_shape))


In [10]:
#Convert preds to labels
reg_labels = preds_to_labels(reg_preds)
noisy_labels = preds_to_labels(noisy_preds)
fgsm_labels = preds_to_labels(fgsm_preds)
bim_a_labels = preds_to_labels(bim_a_preds)
bim_b_labels = preds_to_labels(bim_b_preds)
jsma_labels = preds_to_labels(jsma_preds)

In [11]:
#This is a large network, therefore we need to calculate cos_sim for one type and free memory
grads_train = model.get_gradients_wrt_params(train_data, train_data_labels)
grads_train = normalize(grads_train)
grads_reg_norms, cos_sim_reg = norms_and_cos(model, reg_data, reg_labels, grads_train)
grads_noisy_norms, cos_sim_noisy = norms_and_cos(model, noisy_data, noisy_labels, grads_train)
grads_fgsm_norms, cos_sim_fgsm =norms_and_cos(model, fgsm_data, fgsm_labels, grads_train)
grads_bim_a_norms, cos_sim_bim_a = norms_and_cos(model, bim_a_data, bim_a_labels, grads_train)
grads_bim_b_norms , cos_sim_bim_b= norms_and_cos(model, bim_b_data, bim_b_labels, grads_train)
grads_jsma_norms, cos_sim_jsma = norms_and_cos(model, jsma_data, jsma_labels, grads_train)

../models/util.py:44: RuntimeWarning: invalid value encountered in sqrt
  norms = np.sqrt(np.dot(grads, grads.T)).diagonal()
../models/util.py:33: RuntimeWarning: divide by zero encountered in divide
  matrix_nm[i] = matrix[i]/np.linalg.norm(matrix[i])
../models/util.py:33: RuntimeWarning: invalid value encountered in divide
  matrix_nm[i] = matrix[i]/np.linalg.norm(matrix[i])


In [12]:
#Separate Using Cos Sim
eta = 0.75
print ('Regular: %.4f' % ( greater_cos(cos_sim_reg, eta)))
print ('Noisy:  %.4f' % ( greater_cos(cos_sim_noisy, eta)))
print ('FGSM:  %.4f' % ( greater_cos(cos_sim_fgsm, eta)))
print ('BIM-A:  %.4f' % ( greater_cos(cos_sim_bim_a, eta)))
print ('BIM-B:  %.4f' % ( greater_cos(cos_sim_bim_b, eta)))
print ('JSMA:  %.4f' % ( greater_cos(cos_sim_jsma, eta)))

Regular: 0.8650
Noisy:  0.0000
FGSM:  0.0000
BIM-A:  0.0000
BIM-B:  0.1850
JSMA:  0.3650


In [13]:
#Separate using just norm
gamma = 29
print ('Regular: %.4f' % ( smaller_norm(grads_reg_norms, gamma)))
print ('Noisy:  %.4f' % ( smaller_norm(grads_noisy_norms, gamma)))
print ('FGSM:  %.4f' % ( smaller_norm(grads_fgsm_norms, gamma)))
print ('BIM-A:  %.4f' % ( smaller_norm(grads_bim_a_norms, gamma)))
print ('BIM-B:  %.4f' % ( smaller_norm(grads_bim_b_norms, gamma)))
print ('JSMA:  %.4f' % ( smaller_norm(grads_jsma_norms, gamma)))

Regular: 0.7950
Noisy:  0.7900
FGSM:  1.0000
BIM-A:  1.0000
BIM-B:  0.8150
JSMA:  1.0000


In [14]:
#Use both cos and norm
print ('Regular: %.4f' % ( cos_and_norm_sep(cos_sim_reg, grads_reg_norms, eta, gamma)))
print ('Noisy:  %.4f' % ( cos_and_norm_sep(cos_sim_noisy, grads_noisy_norms, eta, gamma)))
print ('FGSM:  %.4f' % ( cos_and_norm_sep(cos_sim_fgsm, grads_fgsm_norms, eta, gamma)))
print ('BIM-A:  %.4f' % ( cos_and_norm_sep(cos_sim_bim_a, grads_bim_a_norms, eta, gamma)))
print ('BIM-B:  %.4f' % ( cos_and_norm_sep(cos_sim_bim_b, grads_bim_b_norms, eta, gamma)))
print ('JSMA:  %.4f' % ( cos_and_norm_sep(cos_sim_jsma, grads_jsma_norms, eta, gamma)))

Regular: 0.7850
Noisy:  0.0000
FGSM:  0.0000
BIM-A:  0.0000
BIM-B:  0.1250
JSMA:  0.3650


In [15]:
#Get Distortion
print ('Noisy: %.5f' % (avg_l2_dist(reg_data, noisy_data)))
print ('FGSM: %.5f' % (avg_l2_dist(reg_data, fgsm_data)))
print ('BIM-A: %.5f' % (avg_l2_dist(reg_data, bim_a_data)))
print ('BIM-B: %.5f' % (avg_l2_dist(reg_data, bim_b_data)))
print ('JSMA: %.5f' % (avg_l2_dist(reg_data, jsma_data)))

Noisy: 3.16228
FGSM: 542.34713
BIM-A: 1.00000
BIM-B: 1.83570
JSMA: 1.31839


In [16]:
#Ratio of perturbed samples having cos sim greater
print ('Noisy:  %.4f' % ( comp_cos(cos_sim_noisy, cos_sim_reg)))
print ('FGSM:  %.4f' % ( comp_cos(cos_sim_fgsm, cos_sim_reg)))
print ('BIM-A:  %.4f' % ( comp_cos(cos_sim_bim_a, cos_sim_reg)))
print ('BIM-B:  %.4f' % ( comp_cos(cos_sim_bim_b, cos_sim_reg)))
print ('JSMA:  %.4f' % ( comp_cos(cos_sim_jsma, cos_sim_reg)))

Noisy:  0.0000
FGSM:  0.0000
BIM-A:  0.0000
BIM-B:  0.0200
JSMA:  0.0000


In [17]:
#Ratio of perturbed samples having cos sim greater
print ('Noisy:  %.4f' % ( comp_norm(grads_noisy_norms, grads_reg_norms)))
print ('FGSM:  %.4f' % ( comp_norm(grads_fgsm_norms, grads_reg_norms)))
print ('BIM-A:  %.4f' % ( comp_norm(grads_bim_a_norms, grads_reg_norms)))
print ('BIM-B:  %.4f' % ( comp_norm(grads_bim_b_norms, grads_reg_norms)))
print ('JSMA: %.4f' % ( comp_norm(grads_jsma_norms, grads_reg_norms)))

Noisy:  0.3350
FGSM:  0.0100
BIM-A:  0.4250
BIM-B:  0.3750
JSMA: 0.3850
